In [6]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [7]:
import pickle
import random
jeopardy_questions = pickle.load(open("jeopardy_questions.p", "rb"))
double_jeopardy_questions = pickle.load(open("double_jeopardy_questions.p", "rb"))
final_jeopardy_questions = pickle.load(open("final_jeopardy_questions.p", "rb"))
tiebreaker_questions = pickle.load(open("tiebreaker_questions.p", "rb"))

In [18]:
class JeopardyBot:
    
    asked_questions = ["Goof?"]
    exit_commands = ["quit", "bye", "exit", "goodbye", "no"]
    
    def make_exit(self, user_input):
        if user_input.lower() in self.exit_commands:
            return True
        else:
            return False
    
    def chat(self):
        pass
    
    def ask_question(self, questions_round):
        question = "Goof?"
        random_categories = random.choices(list(questions_round), k=3)
        choice = 0
        while int(choice) not in list(range(1, 4)):        
            choice = input("""
Pick a category by typing 1, 2, or 3:
    1. {option_1}
    2. {option_2}
    3. {option_3}
> """.format(
                option_1 = random_categories[0],
                option_2 = random_categories[1],
                option_3 = random_categories[2]
            ))
        while question in self.asked_questions:
            prize, question, right_answer = random.choice(questions_round[random_categories[int(choice) - 1]])
        self.asked_questions.append(question)
        answer = input(question + "\n> ")
        if self.check_answer(answer, right_answer):
            print("Yes! The answer was " + right_answer + ". You win $" + str(int(prize)) + "!")
            return prize
        else:
            print("No, the answer was " + right_answer + ". Sorry!")
            return 0
    
    def check_answer(self, answer, right_answer):
        if fuzz.token_set_ratio(answer, right_answer) > 90:
            return True
        else:
            return False

In [19]:
play_jeopardy = JeopardyBot()
play_jeopardy.ask_question(jeopardy_questions)


Pick a category by typing 1, 2, or 3:
    1. ANCIENT GREECE
    2. CLEAN FREAK
    3. WORLD HISTORY
> 1
According to Collier's Ency., in the 7th cent. this city-state won most of the recorded Olympic victories
> Athens
No, the answer was Sparta. Sorry!


0

In [ ]:
+